In [1]:
import pandas as pd
import numpy as np

In [2]:
xls = pd.ExcelFile('./VSN Categorization Template-Consolidated.xlsx')

df = pd.read_excel(xls)

In [3]:
for x in df.columns:
    if "Question" in x or "Answer" in x:
        continue
    print("column : ",x)
    print(df[x].unique())

column :  Keyword
['Child, Immunization, Common Cold' 'child, Immunization record, school'
 'person, shingles, chikenpox, varicella' 'immunization, Canada'
 'Payment, hepatitis B' 'Child, Immunize, MMR, Pregnancy '
 'Measles, mumps and rubella vaccines' 'Influenza vaccines'
 'Measles, mumps and rubella vaccines, Pregnancy'
 'Infants & young children' 'Adults' 'Chickenpox vaccines'
 'School-age children & teens' 'General immunization'
 'School-age children & teens, Chickenpox vaccines' 'Vaccine ingredients'
 'Where to get immunized' 'International travel, 2 months, immunization'
 nan 'Shingles vaccines']
column :  Who is writing this
['Unknown ' 'Parent' 'Mother' 'Adult' 'Child' 'Elderly' 'Grandparent' nan
 'Adolescent' 'Sibling' 'parent' 'adult' 'grandparent' 'elderly' 'Unknown'
 'mother' 'unknown' 'adolescent']
column :  Subject - Person
['Child' nan 'Infant/baby' 'Adolescent' 'Girl' 'Baby' 'General Public'
 'Adult' 'Newborn' 'Grandchild' 'Family' 'Parent' 'Elderly' 'Unknown '
 'Sibli

In [4]:
undesirables = ["-", ",", "i", " ", "Unknown ","unknown "]
for x in df.columns:
    if "Question" in x or "Answer" in x:
        continue
    df[x] = df[x].replace(undesirables,0)
    df[x] = np.where(df[x]==0, 0, 1)

df.to_csv("temp.csv")

In [5]:
df.columns

Index(['Master Question', 'Master Answer', 'Keyword', 'Who is writing this',
       'Subject - Person', 'Subject 1 - Immunization',
       'Subject 2 - Vaccination / General', 'Disease 1', 'Disease 2',
       'Vaccine 1', 'Vaccine 2', 'Other -condition, symptom etc'],
      dtype='object')

In [6]:
data = []
labels = ['Keyword', 'Who is writing this','Subject - Person', 'Subject 1 - Immunization', 'Subject 2 - Vaccination / General', 'Disease 1', 'Disease 2','Vaccine 1', 'Vaccine 2', 'Other -condition, symptom etc']

for idx, row in df.iterrows():
    try:
        text = row['Master Question'] + row['Master Answer']
        xcel_labels = np.zeros(len(labels),dtype=int)
        
        for idx,x in enumerate(labels):
            xcel_labels[idx] = row[x]

        data.append([text, list(xcel_labels)])
    except:
        print("error")

error


In [7]:
# modules:

import os
import string
import random
import numpy as np
import scipy.sparse as sp
from matplotlib import pyplot as plt
from termcolor import colored
import pandas as pd
from csv import reader
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
    
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/edgarmonis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/edgarmonis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
random.Random(0).shuffle(data)
training_data = data[: 1000]
test_data  = data[1000 :]

training_samples = [x[0] for x in training_data]
training_labels = [x[1] for x in training_data]
test_samples = [x[0] for x in test_data]
test_labels = [x[1] for x in test_data]

In [9]:
# preprocessing step:
def preprocess(sentence):
    return sentence

porter_stemmer_instance = PorterStemmer()

# tokenization step:
def tokenize(preprocessed_sentence):
    # tokenizing sentence:
    token_list = []
    tokens = word_tokenize(preprocessed_sentence)
    for token in tokens:
        # stemming tokens:
        token = porter_stemmer_instance.stem(token)
        # stop-word and punctuation removal:
        if token not in stopwords.words('english') and token not in string.punctuation:
            token_list.append(token)
    return token_list

In [10]:
# defining vectorizer instance for preprocessing, tokenizing, stemming, cleaning input sentences
# and for transforming them to tf-idf representations:
tf_idf_vectorizer = TfidfVectorizer(lowercase = True, preprocessor = preprocess, tokenizer = tokenize, analyzer = 'word')

# training vectorizer while also computing training tf-idf representations:
sparse_tf_idf_training_samples = tf_idf_vectorizer.fit_transform(training_samples)

# computing validation tf-idf representations:
sparse_tf_idf_test_samples = tf_idf_vectorizer.transform(test_samples)

In [11]:
from sklearn.metrics import classification_report

for idx, clss in enumerate(labels):
    print(clss)
    new_training_labels = [x[idx] for x in training_labels]
    new_test_labels = [x[idx] for x in test_labels]
    NBClassifier = ComplementNB()
    NBClassifier.fit(sparse_tf_idf_training_samples, new_training_labels)

    accuracy = NBClassifier.score(sparse_tf_idf_training_samples, new_training_labels)
    print("train accuracy : ", accuracy)

    accuracy = NBClassifier.score(sparse_tf_idf_test_samples, new_test_labels)
    predictions = NBClassifier.predict(sparse_tf_idf_test_samples)
    print(classification_report(new_test_labels,predictions))

Keyword
train accuracy :  1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       460

    accuracy                           1.00       460
   macro avg       1.00      1.00      1.00       460
weighted avg       1.00      1.00      1.00       460

Who is writing this
train accuracy :  0.792
              precision    recall  f1-score   support

           0       0.57      0.09      0.16       129
           1       0.73      0.97      0.84       331

    accuracy                           0.73       460
   macro avg       0.65      0.53      0.50       460
weighted avg       0.69      0.73      0.65       460

Subject - Person
train accuracy :  0.82
              precision    recall  f1-score   support

           0       0.67      0.34      0.45       158
           1       0.73      0.91      0.81       302

    accuracy                           0.72       460
   macro avg       0.70      0.63      0.63       460
weighted avg  

In [12]:
def give_me_k_times_more_negative_than_positive(sample_list, label_list, k):

    temp = list(zip(sample_list[:], label_list[:]))
    random.Random(0).shuffle(temp)
    temp_samples, temp_labels = zip(*temp)

    
    positive = []
    negative = []

    for s, l in zip(temp_samples, temp_labels):
        if l == 1:
            positive.append((s, l))
    
    for s, l in zip(temp_samples, temp_labels):
        if l == 0:
            negative.append((s, l)) 
            if (len(negative) == len(positive) * k):
                break
    
    temp = positive + negative
    random.Random(1).shuffle(temp)
    temp_samples, temp_labels = zip(*temp)
    sparse_temp_samples = tf_idf_vectorizer.transform(temp_samples)
    return sparse_temp_samples, temp_labels

In [13]:
from sklearn.metrics import classification_report

for idx, clss in enumerate(labels):
    print(clss)
    new_training_labels = [x[idx] for x in training_labels]
    new_test_labels = [x[idx] for x in test_labels]
    NBClassifier = ComplementNB()
    new_resampled_training_samples, new_training_labels = give_me_k_times_more_negative_than_positive(training_samples,new_training_labels,1)
    NBClassifier.fit(sparse_tf_idf_training_samples, new_training_labels)

    accuracy = NBClassifier.score(sparse_tf_idf_test_samples, new_test_labels)
    predictions = NBClassifier.predict(sparse_tf_idf_test_samples)
    print(classification_report(new_test_labels,predictions))

Keyword
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       460

    accuracy                           1.00       460
   macro avg       1.00      1.00      1.00       460
weighted avg       1.00      1.00      1.00       460

Who is writing this
              precision    recall  f1-score   support

           0       0.50      0.01      0.02       129
           1       0.72      1.00      0.84       331

    accuracy                           0.72       460
   macro avg       0.61      0.50      0.43       460
weighted avg       0.66      0.72      0.61       460

Subject - Person
              precision    recall  f1-score   support

           0       0.52      0.07      0.12       158
           1       0.67      0.97      0.79       302

    accuracy                           0.66       460
   macro avg       0.59      0.52      0.46       460
weighted avg       0.62      0.66      0.56       460

Subject 1 - Immunization
   

In [14]:
import pickle

In [ ]:
from sklearn.metrics import classification_report


classifier_dict = {}
for idx, clss in enumerate(labels):
    print(clss)
    new_training_labels = [x[idx] for x in training_labels]
    new_test_labels = [x[idx] for x in test_labels]
    NBClassifier = ComplementNB()
    NBClassifier.fit(sparse_tf_idf_training_samples, new_training_labels)

    accuracy = NBClassifier.score(sparse_tf_idf_training_samples, new_training_labels)
    print("train accuracy : ", accuracy)

    accuracy = NBClassifier.score(sparse_tf_idf_test_samples, new_test_labels)
    predictions = NBClassifier.predict(sparse_tf_idf_test_samples)
    print(classification_report(new_test_labels,predictions))
    classifier_dict[clss]=NBClassifier